In [1]:
import pandas as pd
pd.set_option("display.max_columns", 200)

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
input_path = "../../kaggle/input/home-credit-default-risk/" # on local machine
# input_path = "/kaggle/input/home-credit-default-risk/" # on Kaggle

installments_payments = pd.read_csv(input_path + "installments_payments.csv")
POS_CASH_balance = pd.read_csv(input_path + "POS_CASH_balance.csv")
credit_card_balance = pd.read_csv(input_path + "credit_card_balance.csv")


In [3]:
installments_payments.drop(columns="SK_ID_PREV", inplace=True)
POS_CASH_balance.drop(columns="SK_ID_PREV", inplace=True)
credit_card_balance.drop(columns="SK_ID_PREV", inplace=True)

In [4]:
POS_CASH_balance.columns = [col + "_POS" for col in POS_CASH_balance.columns]
credit_card_balance.columns = [col + "_CC" for col in credit_card_balance.columns]

In [5]:
balances_and_payments = pd.merge(installments_payments, credit_card_balance, how="left", left_on="SK_ID_CURR", right_on="SK_ID_CURR_CC")

: 

In [ ]:
balances_and_payments = pd.merge(balances_and_payments, credit_card_balance, how="left", left_on="SK_ID_CURR", right_on="SK_ID_CURR_CC")

In [ ]:
balances_and_payments.drop

In [ ]:
credit_card_balance

In [ ]:
application_train = pd.merge(application_train, bureau, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, previous_application, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, POS_CASH_balance, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])
application_train = pd.merge(application_train, credit_card_balance, how="left", on="SK_ID_CURR")
application_train = application_train.drop_duplicates(subset=["SK_ID_CURR"])

In [ ]:
# saving preprocessor for future use
to_save = {"preprocessor" : preprocessor,
           "porch_func" : porch_func}
filename = "preprocessor.joblib"

joblib.dump(to_save, filename)

In [ ]:
import shap